In [2]:
# coding=utf-8

from datetime import datetime, timedelta
import pandas as pd
from io import StringIO
import requests
import pandahouse

1. Параллельно будем обрабатывать две таблицы. В feed_actions для каждого юзера посчитаем число просмотров и лайков контента. В message_actions для каждого юзера считаем, сколько он получает и отсылает сообщений, скольким людям он пишет, сколько людей пишут ему. Каждая выгрузка должна быть в отдельном таске.

In [85]:
connection = { 
    'host' : 'https://clickhouse.lab.karpov.courses', 
    'password' : 'dpo_python_2020', 
    'user' : 'student', 
    'database' : 'simulator'
}

In [94]:
query = """SELECT  toDate(time) as event_date,
                   countIf(action='like') AS likes,
                   countIf(action='view') AS views,
                   user_id, 
                   gender,
                   os, 
                   age
            FROM simulator_20240620.feed_actions
            WHERE toDate(time) = today() - 1
            GROUP BY user_id, gender, os, age, event_date
            """

df_feed_actions = pandahouse.read_clickhouse(connection=connection, query=query)

ClickhouseException: b"Code: 62. DB::Exception: Syntax error: failed at position 428 ('FORMAT') (line 11, col 33): FORMAT TSVWithNamesAndTypes\n. Expected one of: SETTINGS, end of query. (SYNTAX_ERROR) (version 21.12.2.17 (official build))\n"

In [6]:
df_feed_actions

,event_date,likes,views,user_id,gender,os,age
0,2024-07-24,5,35,84981,1,iOS,20
1,2024-07-24,3,18,2894,0,Android,23
2,2024-07-24,5,22,120965,1,Android,21
3,2024-07-24,4,36,116918,0,iOS,61
4,2024-07-24,9,33,150361,1,Android,35
...,...,...,...,...,...,...,...
18668,2024-07-24,5,17,80515,0,Android,22
18669,2024-07-24,21,83,13571,0,Android,22
18670,2024-07-24,8,62,150668,0,Android,15
18671,2024-07-24,10,41,165987,1,iOS,19


In [87]:
query = """SELECT
                toDate(time) as event_date,
                received.messages_received,
                COUNT(*) AS messages_sent,
                COUNT(DISTINCT receiver_id) AS users_received,
                received.users_sent,
                user_id,
                gender,
                os,
                age
            FROM simulator_20240620.message_actions 
                LEFT JOIN
                (
                    SELECT
                        receiver_id,
                        toDate(time) as event_date,
                        COUNT(*) AS messages_received,
                        COUNT(DISTINCT user_id) AS users_sent
                    FROM simulator_20240620.message_actions
                    WHERE toDate(time) = today() - 1
                    GROUP BY receiver_id, event_date
                ) AS received
                ON
                    message_actions.user_id = received.receiver_id
                WHERE toDate(time) = today() - 1
                GROUP BY user_id, received.messages_received, received.users_sent, event_date, gender, os, age
                ORDER BY user_id"""
df_message_actions = pandahouse.read_clickhouse(connection=connection, query=query)

In [8]:
df_message_actions

,event_date,messages_received,messages_sent,users_received,users_sent,user_id,gender,os,age
0,2024-07-24,2,3,3,2,214,0,Android,21
1,2024-07-24,7,13,13,7,295,1,iOS,27
2,2024-07-24,2,5,5,2,474,1,Android,19
3,2024-07-24,10,4,3,8,494,1,Android,33
4,2024-07-24,2,4,4,2,545,1,iOS,18
...,...,...,...,...,...,...,...,...,...
2205,2024-07-24,0,6,6,0,131516,0,Android,24
2206,2024-07-24,0,5,5,0,131519,0,iOS,31
2207,2024-07-24,7,4,4,7,131522,1,Android,24
2208,2024-07-24,0,3,1,0,131531,0,iOS,40


2. Далее объединяем две таблицы в одну.

In [90]:
merged_df = pd.merge(df_feed_actions, df_message_actions, on=['user_id', 'event_date', 'gender', 'os', 'age'], how='inner')

In [92]:
merged_df = pd.merge(df_feed_actions, df_message_actions, on=['user_id', 'event_date', 'gender', 'os', 'age'], how='outer').dropna()

In [93]:
merged_df

,event_date,likes,views,user_id,gender,os,age,messages_received,messages_sent,users_received,users_sent
14,2024-07-24,19.0,33.0,131027,1,Android,17,0.0,3.0,3.0,0.0
29,2024-07-24,11.0,75.0,129297,0,iOS,19,0.0,5.0,4.0,0.0
63,2024-07-24,5.0,21.0,125318,1,Android,27,8.0,12.0,12.0,6.0
164,2024-07-24,12.0,52.0,107752,0,iOS,27,15.0,9.0,9.0,14.0
286,2024-07-24,10.0,62.0,123164,1,iOS,23,5.0,6.0,6.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...
18393,2024-07-24,18.0,93.0,129010,0,Android,37,1.0,8.0,2.0,1.0
18409,2024-07-24,16.0,90.0,121957,1,iOS,18,5.0,10.0,9.0,5.0
18499,2024-07-24,4.0,31.0,108225,0,Android,44,0.0,5.0,2.0,0.0
18543,2024-07-24,18.0,82.0,122843,0,iOS,27,10.0,6.0,6.0,10.0


In [ ]:
metric_list = []

In [ ]:
full_metric_list = metric_list + ['gender']
group_metric_list = ['event_date'] + ['gender']
df_gender = merged_df[full_metric_list]

In [48]:
df_gender = merged_df[['event_date', 'likes', 'views', 'messages_sent', 'messages_received', 'users_sent', 'users_received']].copy()
df_gender['dimension'] = 'gender'
df_gender['dimension_value'] = merged_df['gender']

df_age = merged_df[['event_date', 'likes', 'views', 'messages_sent', 'messages_received', 'users_sent', 'users_received']].copy()
df_age['dimension'] = 'age'
df_age['dimension_value'] = merged_df['age'].astype(str)

df_os = merged_df[['event_date', 'likes', 'views', 'messages_sent', 'messages_received', 'users_sent', 'users_received']].copy()
df_os['dimension'] = 'os'
df_os['dimension_value'] = merged_df['os']

# Объединяем все DataFrame с помощью pd.concat
df_expanded = pd.concat([df_gender, df_age, df_os], ignore_index=True)

In [14]:
df_expanded

In [49]:
df_expanded

,event_date,likes,views,messages_sent,messages_received,users_sent,users_received,dimension,dimension_value
0,2024-07-24,19,33,3,0,0,3,gender,1
1,2024-07-24,11,75,5,0,0,4,gender,0
2,2024-07-24,5,21,12,8,6,12,gender,1
3,2024-07-24,12,52,9,15,14,9,gender,0
4,2024-07-24,10,62,6,5,5,6,gender,1
...,...,...,...,...,...,...,...,...,...
871,2024-07-24,18,93,8,1,1,2,os,Android
872,2024-07-24,16,90,10,5,5,9,os,iOS
873,2024-07-24,4,31,5,0,0,2,os,Android
874,2024-07-24,18,82,6,10,10,6,os,iOS


3. Для этой таблицы считаем все эти метрики в разрезе по полу, возрасту и ос. Делаем три разных таска на каждый срез

In [50]:
df_os = df_expanded.loc[df_expanded.dimension == 'os'].groupby(['event_date', 'dimension', 'dimension_value'], as_index=False) \
                                              .agg({'views': 'sum', \
                                                    'likes': 'sum', \
                                                    'messages_received': 'sum', \
                                                    'messages_sent': 'sum', \
                                                   'users_received': 'sum',
                                                   'users_sent': 'sum'})


In [51]:
df_gender = df_expanded.loc[df_expanded.dimension == 'gender'].groupby(['event_date', 'dimension', 'dimension_value'], as_index=False) \
                                              .agg({'views': 'sum', \
                                                    'likes': 'sum', \
                                                    'messages_received': 'sum', \
                                                    'messages_sent': 'sum', \
                                                   'users_received': 'sum',
                                                   'users_sent': 'sum'})

df_gender.loc[df_gender['dimension_value'] == 0, 'dimension_value'] = 'female'
df_gender.loc[df_gender['dimension_value'] == 1, 'dimension_value'] = 'male'

In [52]:
df_age = df_expanded.loc[df_expanded.dimension == 'age'].groupby(['event_date', 'dimension', 'dimension_value'], as_index=False) \
                                              .agg({'views': 'sum', \
                                                    'likes': 'sum', \
                                                    'messages_received': 'sum', \
                                                    'messages_sent': 'sum', \
                                                   'users_received': 'sum',
                                                   'users_sent': 'sum'})

In [53]:
df_expanded_1 = pd.concat([df_gender, df_age, df_os], ignore_index=True)

0 - female
1 - male

In [55]:
df_expanded_1.dtypes

event_date           datetime64[ns]
dimension                    object
dimension_value              object
views                        uint64
likes                        uint64
messages_received            uint64
messages_sent                uint64
users_received               uint64
users_sent                   uint64
dtype: object

4. И финальные данные со всеми метриками записываем в отдельную таблицу в ClickHouse.

Дата - event_date

Название среза - dimension

Значение среза - dimension_value

Число просмотров - views

Число лайков - likes

Число полученных сообщений - messages_received

Число отправленных сообщений - messages_sent

От скольких пользователей получили сообщения - users_received

Скольким пользователям отправили сообщение - users_sent

5. Каждый день таблица должна дополняться новыми данными. 

In [ ]:

def ch_insert_df(df, table='test', host='https://clickhouse.lab.karpov.courses', user='student-rw', password='656e2b0c9c'):
    data = df.to_csv(index=False, sep='\t')
    query = f"INSERT INTO {table} FORMAT TabSeparated"
    r = requests.post(host, data=data.encode("utf-8"), auth=(user, password), verify=False)

In [ ]:
connection = { 
    'host' : 'https://clickhouse.lab.karpov.courses', 
    'password' : '656e2b0c9c', 
    'user' : 'student-rw', 
    'database' : 'test'
}

In [17]:
def ch_get_df(query = 'SELECT 1', host = 'https://clickhouse.lab.karpov.courses', user = 'student', password = 'dpo_python_2020'):
    r = requests.post(host, data=query.encode("utf-8"), auth=(user, password), verify=False)
    result = pd.read_csv(StringIO(r.text), sep='\t')
    return result

In [18]:
    def extract_feed_actions():
        query = """SELECT user_id, 
                           toDate(time) as event_date,
                           gender,
                           os, 
                           age,
                           countIf(action='like') AS likes, 
                           countIf(action='view') AS views
                    FROM simulator_20240620.feed_actions
                    WHERE toDate(time) = today() - 1
                    GROUP BY user_id, gender, os, age, event_date """
        df_feed_actions = ch_get_df(query=query)
        return df_feed_actions

In [44]:
def dag_lesson_7_ex_1():
    
    def extract_feed_actions():
        query = """SELECT user_id, 
                           toDate(time) as event_date,
                           gender,
                           os, 
                           age,
                           countIf(action='like') AS likes, 
                           countIf(action='view') AS views
                    FROM simulator_20240620.feed_actions
                    WHERE toDate(time) = today() - 1
                    GROUP BY user_id, gender, os, age, event_date
                    format TSVWithNames"""
        df_feed_actions = ch_get_df(query=query)
        return df_feed_actions
    
    def extract_message_actions():
        query = """SELECT
                        user_id,
                        toDate(time) as event_date,
                        gender,
                        os,
                        age,
                        COUNT(*) AS messages_sent,
                        COUNT(DISTINCT receiver_id) AS users_received,
                        received.messages_received,
                        received.users_sent
                    FROM simulator_20240620.message_actions 
                        LEFT JOIN
                        (
                            SELECT
                                receiver_id,
                                toDate(time) as event_date,
                                COUNT(*) AS messages_received,
                                COUNT(DISTINCT user_id) AS users_sent
                            FROM simulator_20240620.message_actions
                            WHERE toDate(time) = today() - 1
                            GROUP BY receiver_id, event_date
                        ) AS received
                        ON
                            message_actions.user_id = received.receiver_id
                        WHERE toDate(time) = today() - 1
                        GROUP BY user_id, received.messages_received, received.users_sent, event_date, gender, os, age
                        ORDER BY user_id
                        format TSVWithNames"""
        df_message_actions = ch_get_df(query=query)
        return df_message_actions
    
    
    def transform_unite_tables(df_feed_actions, df_message_actions):
        merged_df = pd.merge(df_feed_actions, df_message_actions, on=['user_id', 'event_date', 'gender', 'os', 'age'], how='inner')
        
        df_gender = merged_df[['event_date', 'likes', 'views', 'messages_sent', 'messages_received', 'users_sent', 'users_received']].copy()
        df_gender['dimension'] = 'gender'
        df_gender['dimension_value'] = merged_df['gender']
        
        df_age = merged_df[['event_date', 'likes', 'views', 'messages_sent', 'messages_received', 'users_sent', 'users_received']].copy()
        df_age['dimension'] = 'age'
        df_age['dimension_value'] = merged_df['age'].astype(str)

        df_os = merged_df[['event_date', 'likes', 'views', 'messages_sent', 'messages_received', 'users_sent', 'users_received']].copy()
        df_os['dimension'] = 'os'
        df_os['dimension_value'] = merged_df['os']

        # Объединяем все DataFrame с помощью pd.concat
        df_expanded = pd.concat([df_gender, df_age, df_os], ignore_index=True)
        return df_expanded
    
    df_cube_feed_actions = extract_feed_actions()
    df_cube_message_actions = extract_message_actions()
    
    transform_unite_tables = transform_unite_tables(df_cube_message_actions, df_cube_feed_actions)
    
    return True

dag_lesson_7_ex_1 = dag_lesson_7_ex_1()  

C:\Users\ACER\anaconda3\lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'clickhouse.lab.karpov.courses'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\ACER\anaconda3\lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'clickhouse.lab.karpov.courses'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


C:\Users\ACER\anaconda3\lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'clickhouse.lab.karpov.courses'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\ACER\anaconda3\lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'clickhouse.lab.karpov.courses'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [57]:
connection_2 = { 
    'host' : 'https://clickhouse.lab.karpov.courses', 
    'password' : '656e2b0c9c', 
    'user' : 'student-rw', 
    'database' : 'test'
}

query = """SELECT *
           FROM test.shaimurat_task_7
"""
df_task_7 = pandahouse.read_clickhouse(connection=connection_2, query=query)

ClickhouseException: b"Code: 60. DB::Exception: Table test.shaimurat_task_7 doesn't exist. (UNKNOWN_TABLE) (version 21.12.2.17 (official build))\n"

In [ ]:
#функция работает путём конкатенации
#т.е. она просто присоединит новые данные к низу таблички




In [52]:
from datetime import datetime, timedelta 
import pandas as pd
import pandahouse
from io import StringIO
import requests


connection_1 = { 
    'host' : 'https://clickhouse.lab.karpov.courses', 
    'password' : 'dpo_python_2020', 
    'user' : 'student', 
    'database' : 'simulator'
}

connection_2 = { 
    'host' : 'https://clickhouse.lab.karpov.courses', 
    'password' : '656e2b0c9c', 
    'user' : 'student-rw', 
    'database' : 'test'
}


def dag_lesson_7_ex_1():
    
    def extract_feed_actions():
        query = """SELECT user_id, 
                           toDate(time) as event_date,
                           gender,
                           os, 
                           age,
                           countIf(action='like') AS likes, 
                           countIf(action='view') AS views
                    FROM simulator_20240620.feed_actions
                    WHERE toDate(time) = today() - 1
                    GROUP BY user_id, gender, os, age, event_date
                    """
        df_feed_actions = pandahouse.read_clickhouse(connection=connection_1, query=query)
        return df_feed_actions
    
    def extract_message_actions():
        query = """SELECT
                        user_id,
                        toDate(time) as event_date,
                        gender,
                        os,
                        age,
                        COUNT(*) AS messages_sent,
                        COUNT(DISTINCT receiver_id) AS users_received,
                        received.messages_received,
                        received.users_sent
                    FROM simulator_20240620.message_actions 
                        LEFT JOIN
                        (
                            SELECT
                                receiver_id,
                                toDate(time) as event_date,
                                COUNT(*) AS messages_received,
                                COUNT(DISTINCT user_id) AS users_sent
                            FROM simulator_20240620.message_actions
                            WHERE toDate(time) = today() - 1
                            GROUP BY receiver_id, event_date
                        ) AS received
                        ON
                            message_actions.user_id = received.receiver_id
                        WHERE toDate(time) = today() - 1
                        GROUP BY user_id, received.messages_received, received.users_sent, event_date, gender, os, age
                        ORDER BY user_id
                        """
        df_message_actions = pandahouse.read_clickhouse(connection=connection_1, query=query)
        return df_message_actions    
    
    def transform_unite_tables(df_feed_actions, df_message_actions):
        merged_df = pd.merge(df_feed_actions, df_message_actions, on=['user_id', 'event_date', 'gender', 'os', 'age'], how='inner')
        
        df_gender = merged_df[['event_date', 'likes', 'views', 'messages_sent', 'messages_received', 'users_sent', 'users_received']].copy()
        df_gender['dimension'] = 'gender'
        df_gender['dimension_value'] = merged_df['gender']

        df_age = merged_df[['event_date', 'likes', 'views', 'messages_sent', 'messages_received', 'users_sent', 'users_received']].copy()
        df_age['dimension'] = 'age'
        df_age['dimension_value'] = merged_df['age'].astype(str)

        df_os = merged_df[['event_date', 'likes', 'views', 'messages_sent', 'messages_received', 'users_sent', 'users_received']].copy()
        df_os['dimension'] = 'os'
        df_os['dimension_value'] = merged_df['os']

        # Объединяем все DataFrame с помощью pd.concat
        df_expanded = pd.concat([df_gender, df_age, df_os], ignore_index=True)
        return df_expanded
    
    def transform_age(df_expanded):
        df_expanded_age = df_expanded.loc[df_expanded.dimension == 'age'].groupby(['dimension_value', 'event_date'], as_index=False) \
                                                      .agg({'messages_sent': 'sum', \
                                                           'messages_received': 'sum', \
                                                           'users_sent': 'sum', \
                                                           'users_received': 'sum'})
        return df_expanded_age
    
    def transform_gender(df_expanded):
        df_expanded_gender = df_expanded.loc[df_expanded.dimension == 'gender'].groupby(['dimension_value', 'event_date'], as_index=False) \
                                                      .agg({'messages_sent': 'sum', \
                                                           'messages_received': 'sum', \
                                                           'users_sent': 'sum', \
                                                           'users_received': 'sum'})
        return df_expanded_gender
    
    def transform_os(df_expanded):
        df_expanded_os = df_expanded.loc[df_expanded.dimension == 'os'].groupby(['dimension_value', 'event_date'], as_index=False) \
                                                      .agg({'messages_sent': 'sum', \
                                                           'messages_received': 'sum', \
                                                           'users_sent': 'sum', \
                                                           'users_received': 'sum'})
        return df_expanded_os
    
    def load(df_expanded_age, df_expanded_gender, df_expanded_os):
        df_final = pd.concat([df_expanded_age, df_expanded_gender, df_expanded_os], ignore_index=True)
        return df_final
        
    # Extract data
    df_feed_actions = extract_feed_actions()
    df_message_actions = extract_message_actions()
    
    #Transform data
    df_expanded = transform_unite_tables(df_feed_actions, df_message_actions)
    
    df_expanded_age = transform_age(df_expanded)
    df_expanded_gender = transform_gender(df_expanded)
    df_expanded_os = transform_os(df_expanded)
    
    #Load data
    df_final = load(df_expanded_age, df_expanded_gender, df_expanded_os)
    print(df_final.head())
    return df_final
    
dag_lesson_7_ex_1 = dag_lesson_7_ex_1()      

  dimension_value event_date  messages_sent  messages_received  users_sent  \
0              14 2024-07-23             20                 17          17   
1              15 2024-07-23             29                 13          13   
2              16 2024-07-23             62                 34          31   
3              17 2024-07-23             66                382         143   
4              18 2024-07-23             86                 66          63   

   users_received  
0              11  
1              18  
2              53  
3              64  
4              81  


In [50]:
dag_lesson_7_ex_1

Дата - event_date

Название среза - dimension

Значение среза - dimension_value

Число просмотров - views

Число лайков - likes

Число полученных сообщений - messages_received

Число отправленных сообщений - messages_sent

От скольких пользователей получили сообщения - users_received

Скольким пользователям отправили сообщение - users_sent

In [44]:
        query = """SELECT user_id, 
                           toDate(time) as event_date,
                           gender,
                           os, 
                           age,
                           countIf(action='like') AS likes, 
                           countIf(action='view') AS views
                    FROM simulator_20240620.feed_actions
                    WHERE toDate(time) = today() - 1
                    GROUP BY user_id, gender, os, age, event_date
                    """
        df_feed_actions = pandahouse.read_clickhouse(connection=connection, query=query)

In [45]:
        query = """SELECT
                        user_id,
                        toDate(time) as event_date,
                        gender,
                        os,
                        age,
                        COUNT(*) AS messages_sent,
                        COUNT(DISTINCT receiver_id) AS users_received,
                        received.messages_received,
                        received.users_sent
                    FROM simulator_20240620.message_actions 
                        LEFT JOIN
                        (
                            SELECT
                                receiver_id,
                                toDate(time) as event_date,
                                COUNT(*) AS messages_received,
                                COUNT(DISTINCT user_id) AS users_sent
                            FROM simulator_20240620.message_actions
                            WHERE toDate(time) = today() - 1
                            GROUP BY receiver_id, event_date
                        ) AS received
                        ON
                            message_actions.user_id = received.receiver_id
                        WHERE toDate(time) = today() - 1
                        GROUP BY user_id, received.messages_received, received.users_sent, event_date, gender, os, age
                        ORDER BY user_id
                        """
        df_message_actions = pandahouse.read_clickhouse(connection=connection, query=query)

In [4]:
connection_1 = { 
    'host' : 'https://clickhouse.lab.karpov.courses', 
    'password' : 'dpo_python_2020', 
    'user' : 'student', 
    'database' : 'simulator'
}

In [5]:
        query = """SELECT user_id, 
                           toDate(time) as event_date,
                           gender,
                           os, 
                           age,
                           countIf(action='like') AS likes, 
                           countIf(action='view') AS views
                    FROM simulator_20240620.feed_actions
                    WHERE toDate(time) = today() - 1
                    GROUP BY user_id, gender, os, age, event_date
                    """
        df_feed_actions = pandahouse.read_clickhouse(connection=connection_1, query=query)

In [6]:
        query = """SELECT
                        user_id,
                        toDate(time) as event_date,
                        gender,
                        os,
                        age,
                        COUNT(*) AS messages_sent,
                        COUNT(DISTINCT receiver_id) AS users_received,
                        received.messages_received,
                        received.users_sent
                    FROM simulator_20240620.message_actions 
                        LEFT JOIN
                        (
                            SELECT
                                receiver_id,
                                toDate(time) as event_date,
                                COUNT(*) AS messages_received,
                                COUNT(DISTINCT user_id) AS users_sent
                            FROM simulator_20240620.message_actions
                            WHERE toDate(time) = today() - 1
                            GROUP BY receiver_id, event_date
                        ) AS received
                        ON
                            message_actions.user_id = received.receiver_id
                        WHERE toDate(time) = today() - 1
                        GROUP BY user_id, received.messages_received, received.users_sent, event_date, gender, os, age
                        ORDER BY user_id
                        """
        df_message_actions = pandahouse.read_clickhouse(connection=connection_1, query=query)

In [39]:
        merged_df = pd.merge(df_feed_actions, df_message_actions, on=['user_id', 'event_date', 'gender', 'os', 'age'], how='inner')
        
        df_gender = merged_df[['event_date', 'likes', 'views', 'messages_sent', 'messages_received', 'users_sent', 'users_received']].copy()
        df_gender['dimension'] = 'gender'
        df_gender['dimension_value'] = merged_df['gender'].astype(str)

        df_age = merged_df[['event_date', 'likes', 'views', 'messages_sent', 'messages_received', 'users_sent', 'users_received']].copy()
        df_age['dimension'] = 'age'
        df_age['dimension_value'] = merged_df['age'].astype(str)

        df_os = merged_df[['event_date', 'likes', 'views', 'messages_sent', 'messages_received', 'users_sent', 'users_received']].copy()
        df_os['dimension'] = 'os'
        df_os['dimension_value'] = merged_df['os'].astype(str)

        # Объединяем все DataFrame с помощью pd.concat
        df_expanded = pd.concat([df_gender, df_age, df_os], ignore_index=True)

In [77]:
merged_df.dtypes

user_id                      uint32
event_date           datetime64[ns]
gender                         int8
os                           object
age                           int16
likes                        uint64
views                        uint64
messages_sent                uint64
users_received               uint64
messages_received            uint64
users_sent                   uint64
dtype: object

In [40]:
        df_expanded_age = df_expanded.loc[df_expanded.dimension == 'age'].groupby(['event_date', 'dimension', 'dimension_value'], as_index=False) \
                                              .agg({'views': 'sum', \
                                                    'likes': 'sum', \
                                                    'messages_received': 'sum', \
                                                    'messages_sent': 'sum', \
                                                   'users_received': 'sum',
                                                   'users_sent': 'sum'})

In [41]:
        df_expanded_gender = df_expanded.loc[df_expanded.dimension == 'gender'] \
                                        .groupby(['event_date', 'dimension', 'dimension_value'], as_index=False) \
                                        .agg({'views': 'sum', \
                                              'likes': 'sum', \
                                              'messages_received': 'sum', \
                                              'messages_sent': 'sum', \
                                              'users_received': 'sum',
                                              'users_sent': 'sum'})

        df_expanded_gender.loc[df_expanded_gender['dimension_value'] == 0, 'dimension_value'] = 'female'
        df_expanded_gender.loc[df_expanded_gender['dimension_value'] == 1, 'dimension_value'] = 'male'

In [42]:
        df_expanded_os = df_expanded.loc[df_expanded.dimension == 'os'].groupby(['event_date', 'dimension', 'dimension_value'], as_index=False) \
                                              .agg({'views': 'sum', \
                                                    'likes': 'sum', \
                                                    'messages_received': 'sum', \
                                                    'messages_sent': 'sum', \
                                                   'users_received': 'sum',
                                                   'users_sent': 'sum'})

In [43]:
df_final = pd.concat([df_expanded_age, df_expanded_gender, df_expanded_os], ignore_index=True)

In [44]:
df_final

,event_date,dimension,dimension_value,views,likes,messages_received,messages_sent,users_received,users_sent
0,2024-07-24,age,14,163,34,8,29,10,8
1,2024-07-24,age,15,279,64,30,45,18,29
2,2024-07-24,age,16,388,85,33,64,53,31
3,2024-07-24,age,17,853,199,506,82,80,222
4,2024-07-24,age,18,542,114,57,100,96,57
5,2024-07-24,age,19,945,209,71,116,110,69
6,2024-07-24,age,20,789,164,105,106,104,99
7,2024-07-24,age,21,681,168,378,126,121,202
8,2024-07-24,age,22,371,80,44,39,38,44
9,2024-07-24,age,23,758,154,173,110,108,165


In [75]:
df_final.dtypes

event_date           datetime64[ns]
dimension                    string
dimension_value              string
views                         int32
likes                         int32
messages_received             int32
messages_sent                 int32
users_received                int32
users_sent                    int32
dtype: object

In [ ]:
df_final['event_date'] = df_final['event_date'].astype(Date)

In [74]:
df_final[['dimension', 'dimension_value']] = df_final[['dimension', 'dimension_value']].convert_dtypes()

In [71]:
df_final['dimension'] = df_final['dimension'].astype(str)
df_final['dimension_value'] = df_final['dimension_value'].astype(str)

In [64]:
df_final['dimension'] = df_final['dimension'].astype(str)
df_final['dimension_value'] = df_final['dimension_value'].astype(str)
df_final['views'] = df_final['views'].astype(int)
df_final['likes'] = df_final['likes'].astype(int)
df_final['messages_received'] = df_final['messages_received'].astype(int)
df_final['messages_sent'] = df_final['messages_sent'].astype(int)
df_final['users_received'] = df_final['users_received'].astype(int)
df_final['users_sent'] = df_final['users_sent'].astype(int)

In [30]:
df_final['dimension'] = df_final['dimension'].astype(str)

In [31]:
df_final['dimension_value'] = df_final['dimension_value'].astype(str)

In [95]:
        connection_2 = {
                    'host' : 'https://clickhouse.lab.karpov.courses', 
                    'password' : '656e2b0c9c', 
                    'user' : 'student-rw', 
                    'database' : 'test'
                }
        #запрос создания таблички имеет примерно следующую структуру
        query_test = '''CREATE TABLE IF NOT EXISTS test.shaimuirat_task_7
                        (event_date Date,
                        dimension String,
                        dimension_value String,
                        views Int32,
                        likes Int32,
                        messages_received Int32,
                        messages_sent Int32,
                        users_received Int32,
                        users_sent Int32
                        )
                        ENGINE = MergeTree()
                        ORDER BY event_date
                '''
        pandahouse.execute(query_test, connection=connection_2)
        pandahouse.to_clickhouse(df=df_final, table="shaimuirat_task_7", index=False, connection=connection_2)

ValueError: Unknown type mapping in dtypes: {'event_date': 'DateTime', 'dimension': None, 'dimension_value': None, 'views': 'Int32', 'likes': 'Int32', 'messages_received': 'Int32', 'messages_sent': 'Int32', 'users_received': 'Int32', 'users_sent': 'Int32'}

In [96]:
        connection_2 = {
                    'host' : 'https://clickhouse.lab.karpov.courses', 
                    'password' : '656e2b0c9c', 
                    'user' : 'student-rw', 
                    'database' : 'test'
                }
        #запрос создания таблички имеет примерно следующую структуру
        query_test = '''CREATE TABLE IF NOT EXISTS test.shaimurat_task_7
                        (event_date DateTime,
                        dimension String,
                        dimension_value String,
                        views UInt64,
                        likes UInt64,
                        messages_received UInt64,
                        messages_sent UInt64,
                        users_received UInt64,
                        users_sent UInt64
                        )
                        ENGINE = MergeTree()
                        ORDER BY event_date
                '''
        pandahouse.execute(query_test, connection=connection_2)

b''

In [97]:
pandahouse.to_clickhouse(df=df_final, table="shaimurat_task_7", index=False, connection=connection_2)

ValueError: Unknown type mapping in dtypes: {'event_date': 'DateTime', 'dimension': None, 'dimension_value': None, 'views': 'Int32', 'likes': 'Int32', 'messages_received': 'Int32', 'messages_sent': 'Int32', 'users_received': 'Int32', 'users_sent': 'Int32'}

In [59]:
df_final

,event_date,dimension,dimension_value,views,likes,messages_received,messages_sent,users_received,users_sent
0,2024-07-24,age,14,163,34,8,29,10,8
1,2024-07-24,age,15,279,64,30,45,18,29
2,2024-07-24,age,16,388,85,33,64,53,31
3,2024-07-24,age,17,853,199,506,82,80,222
4,2024-07-24,age,18,542,114,57,100,96,57
5,2024-07-24,age,19,945,209,71,116,110,69
6,2024-07-24,age,20,789,164,105,106,104,99
7,2024-07-24,age,21,681,168,378,126,121,202
8,2024-07-24,age,22,371,80,44,39,38,44
9,2024-07-24,age,23,758,154,173,110,108,165


In [47]:
import pandas as pd

# Пример DataFrame
data = {'column1': ['value1', 'value2', 'value3'],
        'column2': [1, 2, 3]}
df = pd.DataFrame(data)

# Проверка типов данных до преобразования
print(df.dtypes)

df = df.convert_dtypes()
print(df.dtypes)

column1    object
column2     int64
dtype: object
column1    string
column2     Int64
dtype: object


In [83]:
query = """SELECT * FROM test.shaimurat_task_7"""
df = pandahouse.read_clickhouse(connection=connection_2, query=query)

KeyError: 'DateTime64(3)'